# Tutorial 06, case 1b: Poisson problem with distributed control

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Omega} (y - y_d)^2 dx + \frac{\alpha}{2} \int_{\Omega} u^2 dx$$
s.t.
$$\begin{cases}
- \Delta y = f + u     & \text{in } \Omega\\
         y = 1         & \text{on } \partial\Omega
\end{cases}$$

where
$$\begin{align*}
& \Omega               & \text{unit square}\\
& u \in L^2(\Omega)    & \text{control variable}\\
& y \in H^1(\Omega)    & \text{state variable}\\
& \alpha > 0           & \text{penalization parameter}\\
& y_d                  & \text{a piecewise constant desired state}\\
& f                    & \text{forcing term}
\end{align*}$$
using an adjoint formulation solved by a one shot approach.

The test case is from section 5.1 of
```
F. Negri, G. Rozza, A. Manzoni and A. Quarteroni. Reduced Basis Method for Parametrized Elliptic Optimal Control Problems. SIAM Journal on Scientific Computing, 35(5): A2316-A2340, 2013.
```

In [ ]:
import dolfinx.fem
import dolfinx.io
import dolfinx.mesh
import mpi4py
import numpy as np
import petsc4py
import ufl

In [ ]:
import multiphenicsx.fem
import multiphenicsx.io

### Mesh

In [ ]:
with dolfinx.io.XDMFFile(mpi4py.MPI.COMM_WORLD, "data/rectangle.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    boundaries = infile.read_meshtags(mesh, name="boundaries")
boundaries_1 = boundaries.indices[boundaries.values == 1]

In [ ]:
# Define associated measures
dx = ufl.Measure("dx")(subdomain_data=subdomains)

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_entities(mesh, mesh.topology.dim - 1, boundaries_1)

### Function spaces

In [ ]:
Y = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
U = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
Q = Y.clone()

### Trial and test functions

In [ ]:
(y, u, p) = (ufl.TrialFunction(Y), ufl.TrialFunction(U), ufl.TrialFunction(Q))
(z, v, q) = (ufl.TestFunction(Y), ufl.TestFunction(U), ufl.TestFunction(Q))

 ### Problem data

In [ ]:
alpha = 0.01
y_d_1 = 1.0
y_d_2 = 0.6
ff = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0))
bc0 = petsc4py.PETSc.ScalarType(0)
bc1 = petsc4py.PETSc.ScalarType(1)

### Optimality conditions

In [ ]:
a = [[ufl.inner(y, z) * dx, None, ufl.inner(ufl.grad(p), ufl.grad(z)) * dx],
     [None, alpha * ufl.inner(u, v) * dx, - ufl.inner(p, v) * dx],
     [ufl.inner(ufl.grad(y), ufl.grad(q)) * dx, - ufl.inner(u, q) * dx, None]]
f = [ufl.inner(y_d_1, z) * dx(1) + ufl.inner(y_d_2, z) * dx(2),
     None,
     ufl.inner(ff, q) * dx]
a[2][2] = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0)) * ufl.inner(p, q) * dx
f[1] = ufl.inner(dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0)), v) * dx
bdofs_Y_1 = dolfinx.fem.locate_dofs_topological(Y, mesh.topology.dim - 1, boundaries_1)
bdofs_Q_1 = dolfinx.fem.locate_dofs_topological(Q, mesh.topology.dim - 1, boundaries_1)
bc = [dolfinx.fem.DirichletBC(bc1, bdofs_Y_1, Y),
      dolfinx.fem.DirichletBC(bc0, bdofs_Q_1, Q)]

### Solution

In [ ]:
(y, u, p) = (dolfinx.fem.Function(Y), dolfinx.fem.Function(U), dolfinx.fem.Function(Q))

### Cost functional

In [ ]:
J = (0.5 * ufl.inner(y - y_d_1, y - y_d_1) * dx(1) + 0.5 * ufl.inner(y - y_d_2, y - y_d_2) * dx(2)
     + 0.5 * alpha * ufl.inner(u, u) * dx)

### Uncontrolled functional value

In [ ]:
# Extract state forms from the optimality conditions
a_state = ufl.replace(a[2][0], {q: z})
f_state = ufl.replace(f[2], {q: z})
bc_state = [bc[0]]

In [ ]:
# Assemble the linear system for the state
A_state = dolfinx.fem.assemble_matrix(a_state, bcs=bc_state)
A_state.assemble()
F_state = dolfinx.fem.assemble_vector(f_state)
dolfinx.fem.apply_lifting(F_state, [a_state], [bc_state])
F_state.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
dolfinx.fem.set_bc(F_state, bc_state)

In [ ]:
# Solve
ksp = petsc4py.PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A_state)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F_state, y.vector)
y.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

In [ ]:
J_uncontrolled = mesh.comm.allreduce(dolfinx.fem.assemble_scalar(J), op=mpi4py.MPI.SUM)
print("Uncontrolled J =", J_uncontrolled)
assert np.isclose(J_uncontrolled, 0.24)

In [ ]:
multiphenicsx.io.plot_scalar_field(y, "uncontrolled state")

### Optimal control

In [ ]:
# Assemble the block linear system for the optimality conditions
A = dolfinx.fem.assemble_matrix_block(a, bcs=bc)
A.assemble()
F = dolfinx.fem.assemble_vector_block(f, a, bcs=bc)

In [ ]:
# Solve
yup = dolfinx.fem.create_vector_block(f)
ksp = petsc4py.PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, yup)
yup.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with multiphenicsx.fem.BlockVecSubVectorWrapper(yup, [Y.dofmap, U.dofmap, Q.dofmap]) as yup_wrapper:
    for yup_wrapper_local, component in zip(yup_wrapper, (y, u, p)):
        with component.vector.localForm() as component_local:
            component_local[:] = yup_wrapper_local

In [ ]:
J_controlled = mesh.comm.allreduce(dolfinx.fem.assemble_scalar(J), op=mpi4py.MPI.SUM)
print("Optimal J =", J_controlled)
assert np.isclose(J_controlled, 0.158485065)

In [ ]:
multiphenicsx.io.plot_scalar_field(y, "state")

In [ ]:
multiphenicsx.io.plot_scalar_field(u, "control")

In [ ]:
multiphenicsx.io.plot_scalar_field(p, "adjoint")